# Weather Lookup
Dieses Notebook zeigt, wie Wetterdaten für Tripdaten anhand von Breite, Länge und Startzeit abgefragt werden können.

In [ ]:
import pandas as pd
import openmeteo_requests
import requests_cache
from retry_requests import retry
from datetime import datetime
import sys
from pathlib import Path
sys.path.append(str(Path('src').resolve()))
from db import open_connection

cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)


In [ ]:
conn = open_connection()
query = "SELECT id, start_lat, start_lon, start_time FROM public.trips LIMIT 5;"
trips = pd.read_sql_query(query, conn)
conn.close()
trips

In [ ]:
def safe_float(x):    try:        return float(x)    except (TypeError, ValueError):        return Nonedef fetch_weather(lat, lon, timestamp):    ts = pd.to_datetime(timestamp, utc=True)    date = ts.date()    url = 'https://api.open-meteo.com/v1/forecast'    params = {        'latitude': lat,        'longitude': lon,        'start_date': date,        'end_date': date,        'hourly': 'temperature_2m,relativehumidity_2m,precipitation,cloudcover,snowfall,windspeed_10m',        'timezone': 'UTC',    }    response = openmeteo.weather_api(url, params=params)[0]    hourly = response.Hourly()    times = pd.date_range(        start=pd.to_datetime(hourly.Time(), unit='s', utc=True),        end=pd.to_datetime(hourly.TimeEnd(), unit='s', utc=True),        freq=pd.Timedelta(seconds=hourly.Interval()),        inclusive='left'    )    data = pd.DataFrame({        'time': times,        'temperature_2m': hourly.Variables(0).ValuesAsNumpy(),        'relativehumidity_2m': hourly.Variables(1).ValuesAsNumpy(),        'precipitation': hourly.Variables(2).ValuesAsNumpy(),        'cloudcover': hourly.Variables(3).ValuesAsNumpy(),        'snowfall': hourly.Variables(4).ValuesAsNumpy(),        'windspeed_10m': hourly.Variables(5).ValuesAsNumpy(),    })    idx = (data['time'] - ts).abs().argsort().iloc[0]    row = data.iloc[idx]    return {        'temperature': safe_float(row['temperature_2m']),        'humidity': safe_float(row['relativehumidity_2m']),        'precipitation': safe_float(row['precipitation']),        'cloudcover': safe_float(row['cloudcover']),        'snowfall': safe_float(row['snowfall']),        'windspeed': safe_float(row['windspeed_10m']),    }

In [ ]:
weather = trips.apply(lambda r: pd.Series(fetch_weather(r.start_lat, r.start_lon, r.start_time)), axis=1)
trips = pd.concat([trips, weather], axis=1)
trips